<a href="https://colab.research.google.com/github/Flashcl/pantip_scraper/blob/master/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pythainlp
!pip install deepcut

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import re
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp.util import isthai,normalize
import string  
import itertools
import csv
from tqdm import tqdm
import os


In [0]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

rex = re.compile(r'<div class="display-post-story">(.*?)</div>', re.DOTALL)
regexp = re.compile(r'{.*?}')

def cleanAll(raw_html):
  match = [''.join("รร" if "รร" in ''.join(list(j)) else 
                   "แ" if "เเ" in ''.join(list(j)) else i 
            for i, j in itertools.groupby(cleanhtml(i.strip())) )
            .replace("\t\t","\t").replace("\t"," ")
            .replace("\n\n","\n").replace("\n"," ") 
            .replace("  "," ").strip()
            for i in rex.findall(raw_html) if not regexp.search(i)]
  return match

def cleanWord(t):
  c_t = list()
  ck = True
  for word in t:
    if word.strip() and isthai(word) and word not in string.punctuation :
      temp = word in thai_words() or word in "ๆ"
      ck = ck and temp
      c_t.append((word,temp)) 
  return (c_t,ck)

def clean(TID,num,path=""):
  name = str(TID)+".html"
  num = str(num)+"/"
  data = open(path+num+name, "r").read()
  match = cleanAll(data)
  t_list = []
  for text in match:
    t_newmm   = cleanWord(word_tokenize(text,engine = "newmm"))
    t_deepcut = cleanWord(word_tokenize(text,engine = "deepcut"))
    t_attacut = cleanWord(word_tokenize(text,engine = "attacut"))
    d = {"TID":int(name[:-5]), 
         "text":text, 
         "t_newmm":t_newmm, 
         "t_deepcut":t_deepcut, 
         "t_attacut":t_attacut,
         "isInCorpus":  (t_newmm[1]   and t_deepcut[1]) or 
                        (t_deepcut[1] and t_attacut[1]) or 
                        (t_attacut[1] and t_newmm[1])
         }
    t_list.append(d)
  return t_list

def savefile(isInCorpus, num, TID, t_list):
  folder = "InCorpus/" if isInCorpus else "OutCorpus/"
  num = str(num)+"/"
  name = str(TID)+".csv"
  with open(clean_path+folder+num+name, 'w') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(t_list)

In [6]:
%cd "drive/Shared drives/Senior Project/"

/content/drive/Shared drives/Senior Project


In [0]:
raw_path = "raw_html/"
clean_path = "clean_html/"


In [0]:
# change number to change subfolder
num = 1 
cleaned = set([int(s.split('.')[0]) for s in os.listdir(clean_path+"InCorpus/"+str(num)+"/") 
                if len(s.split('.')[0]) > 0 and s.split('.')[0].isdigit() ] )
data = set([int(s.split('.')[0]) for s in os.listdir(raw_path+str(num)+"/") 
                if len(s.split('.')[0]) > 0 and s.split('.')[0].isdigit() ] )
for i in tqdm(data):
  if i not in cleaned :
    # print(i)
    t_list = clean(i, num, raw_path)
    Out = [row for row in t_list if row["isInCorpus"] is False]
    In = [row for row in t_list if row["isInCorpus"] is True]
    savefile(True,num,i,In)
    savefile(False,num,i,Out)

 45%|████▍     | 5996/13412 [05:23<4:15:54,  2.07s/it]

In [0]:
#%mv 3* Other/

In [0]:
#%cd "drive/Shared drives/Senior Project/html/1/"

In [0]:
# %ls

In [0]:
# %mv 3002* ../3/